# Vectorization

Big Picture
- What is vectorization?
- When is it useful?
- What can the compiler do?
- How can you do it manually?


All the code can be found at [https://github.com/wheatman/Vectorization_Notes](https://github.com/wheatman/Vectorization_Notes)

Vectorization is a style of computer programming where operations are applied to multiple elements at the same time instead of individual elements.

Vector code has special registers and special instructions.

Most registers are between 8 and 64 bits

![x86 registers](source/registers.JPG "X86 registers")

Vector registers are between 128 and 512 bits currently.

The vector instructions treat these bits as representing a vector of multiple elements of data and perform the operation element wise.

For example, take a 128 bit register.

We can think of these 128 bits as representing four 32-bit elements.

We can then compare multiple registers each containing these four elements and get the results of all four comparisons at once


![Vector Operation](source/vector_op.JPG "Vector Operation")


The instructions I am describing here is [pcmpeqd](https://www.intel.com/content/www/us/en/docs/intrinsics-guide/index.html#text=_mm_cmpeq_epi32&ig_expand=878)

Formally, given `a` and `b` each of which is a 128 bit register, output dest, a new 128 bit register calculated in the following way

```
FOR j := 0 to 3
	i := j*32
	dst[i+31:i] := ( a[i+31:i] == b[i+31:i] ) ? 0xFFFFFFFF : 0
ENDFOR
```

If you want to operate on 256 bit registers you need to use a different instructions, such as [vpcmpeqd](https://www.intel.com/content/www/us/en/docs/intrinsics-guide/index.html#text=_mm256_cmpeq_epi32&ig_expand=878,879)

Now let's look at a problem.

Given an array of data, we want to count how many ints are equal to the target.

Here is some simple code to count.
```c
long count_ints(int *data, long n, int target) {
  int count = 0;
  for (int i = 0; i < n; i++) {
    if (data[i] == target) {
      count += 1;
    }
  }
  return count;
}
```


To help us look at code and how it compiles we are going [Compiler Explorer](https://godbolt.org/)

This website allows you to, among other things, compile code online and view the assembly.

Lets start by looking at the assembly for this specific problem.

We can see it [https://godbolt.org/z/8debMTGso](https://godbolt.org/z/8debMTGso)

We can see that the compiler is able to do some vectorization for us


Lets run these different versions and see the performance

| Version | time to process 1 million bytes in microseconds |
|--- | --- |
| unoptimized | 172 |
| optimized simple | 98 |
| optimized | 90|
| march=native | 45|


Some notes on compiler options
One of the most important flags is `-O` which specifies the optimization level between `-O0` and `-O3`.  

There are also special versions for optimizing for size `-Os` and `-Oz`.

Lastly there is `-Ofast`  This can make code even faster, but at the price of correctness.  
This mostly deals with floating point numbers, but it is something to be aware of.


I am also sometimes using the flag `-fno-unroll-loops`  This disables loop unrolling and as such is normally not good for performance, but can help make the code smaller.  I use it here only to make the assembly a little easier to read.

Then there is the `-m` flag.  This specifies which architecture to generate code for.  By default most compilers will generate code that will work on a very wide set of cpus, but you can use this flag to specify more specifically what system to run on.  This allows the compiler to generate code which is much faster on some cpus, but will crash other cpus.  

`-mavx2` specifies a cpu which supports the avx2 extension

`-march=native` says just generate code for whatever system I am running the compiler on. 

If you want to see what sort of extra hardware is on your machine you can look with `lscpu` or `cat /proc/cpuinfo`

For example if we look at my laptop 

```
Architecture:            x86_64
  CPU op-mode(s):        32-bit, 64-bit
  Address sizes:         46 bits physical, 48 bits virtual
  Byte Order:            Little Endian
CPU(s):                  20
  On-line CPU(s) list:   0-19
Vendor ID:               GenuineIntel
  Model name:            13th Gen Intel(R) Core(TM) i9-13900H
    CPU family:          6
    Model:               186
    Thread(s) per core:  2
    Core(s) per socket:  10
    Socket(s):           1
    Stepping:            2
    BogoMIPS:            5990.40
    Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology tsc_reliable nonstop_tsc cpuid pni pclmulqdq vmx ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt tsc_deadline_timer aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp ibrs_enhanced tpr_shadow vnmi ept vpid ept_ad fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid rdseed adx smap clflushopt clwb sha_ni xsaveopt xsavec xgetbv1 xsaves avx_vnni umip waitpkg gfni vaes vpclmulqdq rdpid movdiri movdir64b fsrm serialize flush_l1d arch_capabilities
Virtualization features: 
  Virtualization:        VT-x
  Hypervisor vendor:     Microsoft
  Virtualization type:   full
Caches (sum of all):     
  L1d:                   480 KiB (10 instances)
  L1i:                   320 KiB (10 instances)
  L2:                    12.5 MiB (10 instances)
  L3:                    24 MiB (1 instance)
```
Specifically we are looking at the flags

Here is a modern high end intel server

```
Flags:                           
fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid aperfmperf tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt tsc_deadline_timer aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp ibrs_enhanced fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid avx512f avx512dq rdseed adx smap avx512ifma clflushopt clwb avx512cd sha_ni avx512bw avx512vl xsaveopt xsavec xgetbv1 xsaves wbnoinvd ida arat avx512vbmi pku ospke avx512_vbmi2 gfni vaes vpclmulqdq avx512_vnni avx512_bitalg tme avx512_vpopcntdq rdpid md_clear flush_l1d arch_capabilities
```

Now the flag names are a bit weird

 - SSE (Streaming SIMD Extensions)
 - AVX (Advanced Vector Extensions)


SIMD stands for single instruction multiple data, which means we have a single instruction which processes multiple pieces of data at the same time
AVX is basically just the successor to SIMD

These both allow us to use vectorization 

Now lets try and understand how the vectorized code works

```assembly
count_ints(int*, long, int):                     # @count_ints(int*, long, int)
        xor     eax, eax
        test    rsi, rsi
        jle     .LBB0_8
        cmp     rsi, 8
        jae     .LBB0_4
        xor     ecx, ecx
        jmp     .LBB0_3
.LBB0_4:                                # %vector.ph
        mov     rcx, rsi
        and     rcx, -8
        vmovd   xmm0, edx
        vpbroadcastd    ymm1, xmm0
        vpxor   xmm0, xmm0, xmm0
        xor     eax, eax
.LBB0_5:                                # %vector.body
        vpcmpeqd        ymm2, ymm1, ymmword ptr [rdi + 4*rax]
        vpsubd  ymm0, ymm0, ymm2
        add     rax, 8
        cmp     rcx, rax
        jne     .LBB0_5
        vextracti128    xmm1, ymm0, 1
        vpaddd  xmm0, xmm0, xmm1
        vpshufd xmm1, xmm0, 238                 # xmm1 = xmm0[2,3,2,3]
        vpaddd  xmm0, xmm0, xmm1
        vpshufd xmm1, xmm0, 85                  # xmm1 = xmm0[1,1,1,1]
        vpaddd  xmm0, xmm0, xmm1
        vmovd   eax, xmm0
        cmp     rcx, rsi
        je      .LBB0_7
.LBB0_3:                                # %for.body
        xor     r8d, r8d
        cmp     dword ptr [rdi + 4*rcx], edx
        sete    r8b
        add     eax, r8d
        inc     rcx
        cmp     rsi, rcx
        jne     .LBB0_3
.LBB0_7:                                # %for.cond.cleanup.loopexit
        mov     eax, eax
.LBB0_8:                                # %for.cond.cleanup
        vzeroupper
        ret
```

```c++
long count_ints(int * data, long n, int target) {
    int count = 0;
    int i = 0;
    if (n > 8) {
        int local_end = n - 8;
        vector<int> vector_target = broadcast(target)
        vector<int> vector_count = {0}
        while (i < local_end) {
            vector_count -= vector_equals(*(data+i), vector_target);
            i+=8;
        }
        count += reduce_add(vector_count)
    }
    for (; i < n; i++) {
        if (data[i] == target) {
            count += 1;
        }
  }
  return count;
}

long count_ints(int* data, long n, int target) {
        int count = 0;
        long clean_end = (n / 8) * 8;
        long i = 0;
        if (n > 8) {
                int local_end = n - 8;
                auto vector_target = _mm256_set1_epi32(target);
                auto counts = _mm256_setzero_si256();
                while (i < local_end) {
                        auto vector_compare = _mm256_cmpeq_epi32(
                                        _mm256_loadu_si256((__m256i *)(data + i)), vector_target)
                        vector_count = _mm256_sub_epi32(counts, vector_compare);
                        i+=8;
                }
                count += _mm256_extract_epi32(counts, 0);
                count += _mm256_extract_epi32(counts, 1);
                count += _mm256_extract_epi32(counts, 2);
                count += _mm256_extract_epi32(counts, 3);
                count += _mm256_extract_epi32(counts, 4);
                count += _mm256_extract_epi32(counts, 5);
                count += _mm256_extract_epi32(counts, 6);
                count += _mm256_extract_epi32(counts, 7);
        }=
        for (; i < n; i++) {
                if (data[i] == target) {
                        count += 1;
                }
        }
        return count;
}

```

Now let's make a minor change to the program.

```c++
long count_ints(int* data, long n, int target) {
    // change the type from an int to a long
    long count = 0;
    for (int i = 0; i < n; i++) {
        if (data[i] == target) {
            count+=1;
        }
    }
    return count;
}
```
We changed the type of the count variable from an int to a long

This allows us to correctly count more than 2^31 elements

We can see the new assembly [https://godbolt.org/z/sTdoK9oTr](https://godbolt.org/z/sTdoK9oTr)

We can see how the non vectorized code does not change in any substantial way, but the vectorized code does change the inner loop fundamentally.



The inner loop with an int count variable
```
.LBB0_5:                                # %vector.body
        vpcmpeqd        ymm2, ymm1, ymmword ptr [rdi + 4*rax]
        vpsubd  ymm0, ymm0, ymm2
        add     rax, 8
        cmp     rcx, rax
        jne     .LBB0_5
```

The major issue is that the original approach was actually using the fact that two independant types were actually the same type.  That is to say the count, and the elements being compared.

If the elements being compared are smaller than the count variable then you need to deal with overflow since the original approach uses these same bit positions to store the count distributed in a vector register.


The inner loop with a long count variable 


```
        vpcmpeqd        xmm3, xmm1, xmmword ptr [rdi + 4*rax]
        vpmovzxdq       ymm3, xmm3              # ymm3 = xmm3[0],zero,xmm3[1],zero,xmm3[2],zero,xmm3[3],zero
        vpand   ymm3, ymm3, ymm2
        vpaddq  ymm0, ymm0, ymm3
        add     rax, 4
        cmp     rcx, rax
        jne     .LBB0_6
```

We see it here using `vpmovzxdq` which expands out 4 32 bit elements into 64 bit elements to then combine into 4 64 bit counts which are then large enough.

While this approach makes sense for transforming ints to longs, does it still make sense if we are instead looking for bytes.  In that case we could compare up to 32 bytes at a time, but we will still be limited to only doing the 4 longs that fit into a 256 bit vector.





Now let's look at a second problem.

Here we get an array of bytes, and we want to know how many times the target bytes appears next to itself.  

```c
uint64_t count_pairs(uint8_t *data, uint64_t size, uint8_t target) {
  uint64_t total = 0;
  uint16_t check = target | (target << 8U);
  for (uint64_t i = 0; i < size * 2 - 1; i++) {
    total += (load16(data + i) == check);
  }
  return total;
}
```

This problem seems very similar to the one we looked at before

However, the fact that we are looking at unaligned data makes it a fair amount messier.

First let's take a look at the performance of this code

| Version | time to process  2 million bytes in microseconds |
|--- | --- |
| unoptimized | 2957 |
| optimized simple | 489 |
| optimized | 481 |
| march=native | 513 |

We notice that we are not able to see much speedups with `-march=native`

Lets take a look at the code

[https://godbolt.org/z/8Yasv3vW1](https://godbolt.org/z/8Yasv3vW1)

The vectorized loop is messy as it tries to rearrange  the data to fit into a standard pattern it knows how to vectorize.

Now we have a few options.  The first option is to convert the code into something that is easier to vectorize.  Now we know it can handle standard shorts, so we can just run through the loop twice, once for each alignment.

```c
uint64_t __attribute__((noinline))
count_pairs(uint8_t *data, uint64_t size, uint8_t target) {
  uint64_t total = 0;
  uint16_t check = target | (target << 8U);
  for (uint64_t i = 0; i < size * 2 - 1; i+=2) {
    if (load16(data + i) == check) {
      total +=1;
    }
  }
  for (uint64_t i = 1; i < size * 2 - 1; i+=2) {
    if (load16(data + i) == check) {
      total +=1;
    }
  }
  return total;
}
```

Now the compiler should be able to easily vectorize both loops individually
We can take a look at the code generated [https://godbolt.org/z/9T97PPn46](https://godbolt.org/z/9T97PPn46)

| Version | time to process  2 million bytes in microseconds |
|--- | --- |
| unoptimized | 2957 |
| optimized simple | 489 |
| optimized | 481 |
| march-native | 513 |
| split loops | 346 | 


# Intrinsics

The compiler can do a lot to optimize and use the vector registers to speed up the code, but we can also do it manually.

Intrinsics are basically functions which will compile down to a single special assembly instruction that we can use to use the vector instructions in c or c++ code without having to write raw assembly.  The list of them can be found [https://software.intel.com/sites/landingpage/IntrinsicsGuide/](https://software.intel.com/sites/landingpage/IntrinsicsGuide/)

![_mm256_cmpeq_epi16](source/_mm256_cmpeq_epi16.JPG "_mm256_cmpeq_epi16")




However, we can still do it by hand.

```c++

unsigned long count_pairs(unsigned char *data, unsigned long size, unsigned char target) {
  unsigned long total = 0;
  unsigned short check = target | (target << 8U);
  __m256i compare = _mm256_set1_epi16(target);
  for (uint64_t i = 0; i < size * 2; i += 32) {
    uint32_t block1 = _mm256_movemask_epi8(
        _mm256_cmpeq_epi16(_mm256_load_si256((__m256i *)(data + i)), compare));
    uint32_t block2 = _mm256_movemask_epi8(_mm256_cmpeq_epi16(
        _mm256_loadu_si256((__m256i *)(data + i + 1)), compare));
    total += __builtin_popcount(block1);
    total += __builtin_popcount(block2);
  }
  return total / 2;
}
```

This approach does a similar thing to the idea above in splitting the loops to check the two alignments, but in this case we do both steps at the same time.

Lets walk through this code together

we use 
`_mm256_load_si256((__m256i *)(data + i))` and `_mm256_loadu_si256((__m256i *)(data + i + 1))` to load the two blocks of data, offset from each other by 1 bytes


then we compare each of these to the vector with the comparison with `_mm256_cmpeq_epi16`  

This, as described above, sets all of the bits if the corresponding positions are equal.

Then we use `_mm256_movemask_epi8` to pull the result of the comparison from vector registers back to normal registers.  This instruction gives us the top bit from each byte.

Lastly we just count how many bits are set in each of our comparisons with `__builtin_popcount`

Now since we pulled out bytes, even though we compared by shorts we need to divide by 2 at the end.

So altogether we get a performance of

| Version | time to process  2 million bytes in microseconds |
|--- | --- |
| unoptimized | 2957 |
| optimized simple | 489 |
| optimized | 481 |
| march-native | 513 |
| split loops | 346 | 
| manual vectorization | 117 |





# Summary 

- vectorization can help us speed up our code
- the compiler can help us with many forms of vectorization
- when the compiler can't help us we can do it manually using intrinsics

